In [0]:
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60107031.dfs.core.windows.net",
    "uu8nYhaDL3GsV9lth+OpwIKB+K5rK30hFzynvbAjiDKlZIeJ+GAboPBrtwmXoRCPLAP446WZiO+q+AStNlv5NQ=="
)

In [0]:
authors = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60107031.dfs.core.windows.net/processed/authors/"
)

In [0]:
reviews_clean = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60107031.dfs.core.windows.net/processed/reviews/"
)

In [0]:
books = spark.read.parquet(
    "abfss://lakehouse@goodreadsreviews60107031.dfs.core.windows.net/processed/books/"
)

In [0]:
books.show(3)
authors.show(3)
reviews_clean.show()

+----------+------------------+------------+-------------+-----+-------+----------+--------+--------------+-----------+--------------------+---------+--------------------+---------+----+--------------------+---------+---------------+-------------+-----------------+-------------------+----------------+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+
|      isbn|text_reviews_count|country_code|language_code|count|   name|      asin|is_ebook|average_rating|kindle_asin|         description|   format|                link|author_id|role|           publisher|num_pages|publication_day|       isbn13|publication_month|edition_information|publication_year|                 url|           image_url|book_id|ratings_count|work_id|               title|title_without_series|
+----------+------------------+------------+-------------+-----+-------+----------+--------+--------------+-----------+--------------------+---------+----------------

In [0]:
reviews_clean.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- rating: long (nullable = true)
 |-- review_text: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- date_updated: string (nullable = true)
 |-- read_at: string (nullable = true)
 |-- started_at: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- n_comments: long (nullable = true)



In [0]:
books.printSchema()

root
 |-- isbn: string (nullable = true)
 |-- text_reviews_count: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- count: string (nullable = true)
 |-- name: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- is_ebook: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- kindle_asin: string (nullable = true)
 |-- description: string (nullable = true)
 |-- format: string (nullable = true)
 |-- link: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- role: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- num_pages: string (nullable = true)
 |-- publication_day: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- publication_month: string (nullable = true)
 |-- edition_information: string (nullable = true)
 |-- publication_year: string (nullable = true)
 |-- url: string (nullable = true)
 |-- image_url: string (nullable = tru

In [0]:
authors.printSchema()

root
 |-- average_rating: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- text_reviews_count: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ratings_count: string (nullable = true)



In [0]:
# Bridge between Books and Authors

from pyspark.sql.functions import col

# Assign aliases
b = books.alias("b")
a = authors.alias("a")

# Join books with authors using author_id
book_author_bridge = b.join(a, on="author_id", how="inner")

# Select key columns to view
book_author_bridge = book_author_bridge.select(
    col("b.book_id"),
    col("b.title"),
    col("a.author_id"),
    col("a.name")
)

# Display schema and sample rows
print("Book-Author Bridge Schema:")
book_author_bridge.printSchema()

print("Book-Author Bridge Sample Rows:")
book_author_bridge.show(5, truncate=False)


Book-Author Bridge Schema:
root
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)

Book-Author Bridge Sample Rows:
+-------+--------------------------------------------------------------------+---------+----------------+
|book_id|title                                                               |author_id|name            |
+-------+--------------------------------------------------------------------+---------+----------------+
|5333265|W.C. Fields: A Life on Film                                         |604031   |Ronald J. Fields|
|1333909|Good Harbor                                                         |626222   |Anita Diamant   |
|7327624|The Unschooled Wizard (Sun Wolf and Starhawk, #1-2)                 |10333    |Barbara Hambly  |
|6066819|Best Friends Forever                                                |9212     |Jennifer Weiner |
|287140 |Runic Astrology: Starcraft and 

In [0]:
# Homework Part 1 - Curate the Gold Table

from pyspark.sql.functions import col, lit

# Assign aliases to DataFrames for easier reference
ba = book_author_bridge.alias("ba")
r = reviews_clean.alias("r")  # or reviews_verified if that's your name

# Join the reviews with the book-author bridge on book_id
curated = r.join(ba, on="book_id", how="inner")

# Select only existing columns
curated_reviews_gold = curated.select(
    col("r.review_id"),
    col("r.book_id"),
    col("ba.title"),
    col("ba.author_id"),
    col("ba.name"),
    col("r.user_id"),
    col("r.rating"),
    col("r.review_text"),
    # Fill in missing columns manually with placeholders
    lit(None).alias("language"),
    lit(None).alias("n_votes"),
    lit(None).alias("date_added")
)

# Print the schema and show sample rows
print("Curated Reviews Gold DataFrame Schema:")
curated_reviews_gold.printSchema()

print("Curated Reviews Gold DataFrame Sample Rows:")
curated_reviews_gold.show(5, truncate=False)


Curated Reviews Gold DataFrame Schema:
root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: long (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: void (nullable = true)
 |-- n_votes: void (nullable = true)
 |-- date_added: void (nullable = true)

Curated Reviews Gold DataFrame Sample Rows:
+--------------------------------+--------+------------------------------------------------+---------+-------------+--------------------------------+------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# Use your working catalog and schema
spark.sql("USE CATALOG main")        # Optional, if you have multiple catalogs
spark.sql("USE SCHEMA default")      # Use your schema (you can change it)

# Save the curated DataFrame as a managed Delta table
curated_reviews_gold.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.curated_reviews")

print("✅ Gold table 'curated_reviews' saved and registered successfully.")


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5267100159574389>, line 2
      1 # Use your working catalog and schema
----> 2 spark.sql("USE CATALOG main")        # Optional, if you have multiple catalogs
      3 spark.sql("USE SCHEMA default")      # Use your schema (you can change it)
      5 # Save the curated DataFrame as a managed Delta table

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/session.py:1868, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1863     else:
   1864         raise PySpa

In [0]:
# ✅ Use only schema (default)
spark.sql("USE SCHEMA default")

# Save your curated dataframe as a Delta table
curated_reviews_gold.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.curated_reviews")

print("✅ Gold table 'curated_reviews' saved and registered successfully.")


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-5267100159574390>, line 5
      2 spark.sql("USE SCHEMA default")
      4 # Save your curated dataframe as a Delta table
----> 5 curated_reviews_gold.write \
      6     .format("delta") \
      7     .mode("overwrite") \
      8     .saveAsTable("default.curated_reviews")
     10 print("✅ Gold table 'curated_reviews' saved and registered successfully.")

NameError: name 'curated_reviews_gold' is not defined

In [0]:
# Select the default schema (catalog not needed in community or normal Databricks)
spark.sql("USE SCHEMA default")


DataFrame[]

In [0]:
# Save as a managed Delta table
curated_reviews_gold.write.format("delta").mode("overwrite").saveAsTable("default.curated_reviews_gold")

print("✅ Gold table saved successfully as: default.curated_reviews_gold")


✅ Gold table saved successfully as: default.curated_reviews_gold


In [0]:
# Run a simple SQL query to check number of rows
spark.sql("SELECT COUNT(*) AS total_rows FROM default.curated_reviews_gold").show()


+----------+
|total_rows|
+----------+
|  15739524|
+----------+



In [0]:
spark.sql("""
    SELECT title, name, rating, review_text
    FROM default.curated_reviews_gold
    ORDER BY rating DESC
    LIMIT 10
""").show(truncate=False)


+---------------------------------------+-------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
spark.sql("DESCRIBE TABLE default.curated_reviews_gold").show(truncate=False)


+-----------+---------+-------+
|col_name   |data_type|comment|
+-----------+---------+-------+
|review_id  |string   |NULL   |
|book_id    |string   |NULL   |
|title      |string   |NULL   |
|author_id  |string   |NULL   |
|name       |string   |NULL   |
|user_id    |string   |NULL   |
|rating     |bigint   |NULL   |
|review_text|string   |NULL   |
|language   |void     |NULL   |
|n_votes    |void     |NULL   |
|date_added |void     |NULL   |
+-----------+---------+-------+



In [0]:
# Load the existing gold table
spark.catalog.refreshTable("default.curated_reviews_gold")
gold_df = spark.table("default.curated_reviews_gold")

# Clear any cached metadata for the table
spark.catalog.uncacheTable("default.curated_reviews_gold")
spark.catalog.refreshTable("default.curated_reviews_gold")

# Read directly from the Delta files instead of the catalog, to avoid metadata mismatch
gold_df = spark.read.format("delta").load("abfss://lakehouse@goodreadsreviews60107031.dfs.core.windows.net/gold/curated_reviews")

print("✅ Loaded curated_reviews_gold from Delta path successfully")
gold_df.show(5, truncate=False)




✅ Loaded curated_reviews_gold from Delta path successfully
+--------------------------------+--------+-------------------------------+-------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql.functions import col, split, size

# Add a column for review length in words
gold_features = gold_df.withColumn(
    "review_length_words",
    size(split(col("review_text"), " "))
)

print("✅ Added review_length_words column")
gold_features.select("review_id", "review_length_words").show(5)


✅ Added review_length_words column
+--------------------+-------------------+
|           review_id|review_length_words|
+--------------------+-------------------+
|1ae481c00de2f9bb4...|                 87|
|f91748f3327af0d5d...|                678|
|40fcb1afcdb47ec6f...|                 33|
|36335507e25041a27...|                314|
|1a7d64bcf9b96d7c9...|                  6|
+--------------------+-------------------+
only showing top 5 rows


In [0]:
from pyspark.sql.functions import avg, count

# Aggregate to compute average rating and number of reviews per book
book_features = gold_features.groupBy("book_id").agg(
    avg("rating").alias("avg_rating"),
    count("review_id").alias("num_reviews")
)

print("✅ Computed average rating and number of reviews per book")
book_features.show(5)


✅ Computed average rating and number of reviews per book
+--------+-----------------+-----------+
| book_id|       avg_rating|num_reviews|
+--------+-----------------+-----------+
| 8667848| 3.63249063670412|       4272|
| 1398102|              4.5|          2|
|    6708|3.893030794165316|        617|
|26221746|              4.5|          4|
| 2282133|4.257453857075248|       2113|
+--------+-----------------+-----------+
only showing top 5 rows


In [0]:
# Join back so each review has book-level info
enriched_gold = gold_features.join(book_features, on="book_id", how="left")

print("✅ Joined features to main dataset")
enriched_gold.show(5, truncate=False)


✅ Joined features to main dataset
+--------+--------------------------------+-------------------------------+-------+--------------------------------+------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
output_path = "abfss://lakehouse@goodreadsreviews60107031.dfs.core.windows.net/gold/features_v1"

# Save as Delta table
enriched_gold.write.format("delta").mode("overwrite").save(output_path)

print("✅ Saved enriched dataset to Gold layer at:", output_path)


✅ Saved enriched dataset to Gold layer at: abfss://lakehouse@goodreadsreviews60107031.dfs.core.windows.net/gold/features_v1


In [0]:
# Reload and check schema
features_df = spark.read.format("delta").load(output_path)

print("✅ Reloaded features_v1")
features_df.printSchema()
features_df.show(5, truncate=False)

# Check record count
print("Total records:", features_df.count())


✅ Reloaded features_v1
root
 |-- book_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- review_length_words: integer (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- num_reviews: long (nullable = true)

+--------+--------------------------------+-------------------------------+-------+--------------------------------+------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------